In [0]:
import argparse
import pickle
# from gensim.corpora import Dictionary
# from data_utils import parse_xml, clean_txt
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm
import string
from bs4 import BeautifulSoup as Soup
import numpy as np
from google.colab import drive
import re
from nltk.stem.porter import PorterStemmer
import string
import torch
from urllib.parse import urlparse
import numpy as np
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive/dataset/dataset"
dataset = '/content/drive/My Drive/dataset/dataset'

articles-training-bypublisher-20181122.xml
articles-validation-bypublisher-20181122.xml
article.xsd
ground-truth-training-bypublisher-20181122.xml
ground-truth-validation-bypublisher-20181122.xml
ground-truth.xsd
preprocessed_new_train.pickle
preprocessed_new_val.pickle


In [0]:
FLAGS = re.MULTILINE | re.DOTALL
def re_sub(pattern, repl, text, flags=None):
    if flags is None:
        return re.sub(pattern, repl, text, flags=FLAGS)
    else:
        return re.sub(pattern, repl, text, flags=(FLAGS | flags))

In [0]:
def clean_txt(text,
              remove_stopwords=False, 
              remove_nonalphanumeric=True,
              use_number_special_token=True,
              remove_numbers=False):
    
    text = text.lower()
    text = re.sub(r"-", " ", text)
    text = re.sub(r"[a-zA-Z]+\/[a-zA-Z]+", " ", text)
    text = re.sub(r"\n", " ", text)
    if use_number_special_token:
        text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "numberplaceholder", text)  
    elif remove_numbers:
        text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "", text)
    
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if w not in stops]
        text = " ".join(text)
    
    # Remove URL
    text = re_sub(r"(http)\S+", "", text)
    text = re_sub(r"(www)\S+", "", text)
    text = re_sub(r"(href)\S+", "", text)
    # Remove multiple spaces
    text = re_sub(r"[ \s\t\n]+", " ", text)
    
    # remove repetition
    text = re_sub(r"([!?.]){2,}", r"\1", text)
    text = re_sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2", text)

    return text.strip()

In [0]:
def parse_xml(article):
    # get meta data
    id = article.get('id')
    date = article.get('published-at')
    title = article.get('title')
    title = clean_txt(title)

    # get external link info
    external_links, internal_count=[],0
    for link in article.find_all('a'):
        if str(link.get('type'))=='internal':
            internal_count+=1
        else:
            external_links.append(link.get('href'))

    # get actual text
    article_text = article.get_text()
    article_text = clean_txt(article_text)
    
    result = [id, {'date':date,'title':title, 'internal':internal_count, 'external':external_links, 'article_text':article_text}]
    return result

In [0]:
def parse_label_xml(article):
    id = article.get('id')
    label = article.get('hyperpartisan')
    bias = article.get('bias')
    result = [id, {'hyperpartisan':label, 'bias': bias}]
    return result

In [0]:
def new_data_cleaning():
    for phase in ['val','train']:
        preprocessed_dict = {}
        if phase == 'train':
          base_path = dataset + '/articles-training-bypublisher-20181122.xml'
        else:
          base_path = dataset + '/articles-validation-bypublisher-20181122.xml'
        
        print("Loading data from xml")
        xml_file = open(base_path).read()
        soup = Soup(xml_file,"xml")
        articles=soup.find_all('article')
        print("Number of articles: ", len(articles))

        for a in tqdm(articles):
            id, article = parse_xml(a)
            preprocessed_dict[id] = article

        with open('/content/drive/My Drive/dataset/dataset/preprocessed_new_{}.pickle'.format(phase), 'wb') as handle:
            pickle.dump(preprocessed_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
def label_extraction():
    for phase in ['val', 'train']:
        label_dict = {}
        if phase == 'train':
          base_path = dataset + '/ground-truth-training-bypublisher-20181122.xml'
        else:
          base_path = dataset + '/ground-truth-validation-bypublisher-20181122.xml'
        
        print("Loading ground truth labels")
        xml_file = open(base_path).read()
        soup = Soup(xml_file,'xml')
        articles = soup.find_all('article')
        print('Number of articles with labels:', len(articles))

        for a in tqdm(articles):
            id, article = parse_label_xml(a)
            label_dict[id] = article
        
        with open(dataset + '/ground-truth-{}.pickle'.format(phase),'wb') as handle:
            pickle.dump(label_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
new_data_cleaning()

Loading data from xml


  0%|          | 38/150000 [00:00<06:38, 376.65it/s]

Number of articles:  150000


100%|██████████| 150000/150000 [05:06<00:00, 488.62it/s]


Loading data from xml


In [0]:
label_extraction()

Loading ground truth labels


 29%|██▉       | 44078/150000 [00:00<00:00, 440779.37it/s]

Number of articles with labels: 150000


100%|██████████| 150000/150000 [00:00<00:00, 447901.08it/s]


Loading ground truth labels


  4%|▍         | 23302/600000 [00:00<00:02, 233019.11it/s]

Number of articles with labels: 600000


100%|██████████| 600000/600000 [00:01<00:00, 429064.98it/s]


In [0]:
val_data_path = '/content/drive/My Drive/dataset/dataset/preprocessed_new_val.pickle'
pickle_val = open(val_data_path, 'rb')
val_data = pickle.load(pickle_val)
pickle_val.close()
print(val_data['0000017'])

{'date': '2004-07-29', 'title': 'san francisco / head of juvenile probation department under fire / gas leaks, criticism over reform progress may doom tucker', 'internal': 1, 'external': ['/search/?action=search&amp;channel=politics&amp;inlineLink=1&amp;searchindex=solr&amp;query=%22Gwendolyn+Tucker%22', '/search/?action=search&amp;channel=politics&amp;inlineLink=1&amp;searchindex=solr&amp;query=%22Juvenile+Probation+Commission%22', '/search/?action=search&amp;channel=politics&amp;inlineLink=1&amp;searchindex=solr&amp;query=%22Jesse+Williams%22', '/search/?action=search&amp;channel=politics&amp;inlineLink=1&amp;searchindex=solr&amp;query=%22Gregg+Adam%22', '/search/?action=search&amp;channel=politics&amp;inlineLink=1&amp;searchindex=solr&amp;query=%22Juvenile+Probation+Department%22', '/search/?action=search&amp;channel=politics&amp;inlineLink=1&amp;searchindex=solr&amp;query=%22Andrea+Shorter%22', '/search/?action=search&amp;channel=politics&amp;inlineLink=1&amp;searchindex=solr&amp;q

In [0]:
val_label_path = '/content/drive/My Drive/dataset/dataset/ground-truth-val.pickle'
pickle_val = open(val_label_path, 'rb')
val_label = pickle.load(pickle_val)
pickle_val.close()
print(val_label['0000017'])

{'hyperpartisan': 'false', 'bias': 'left-center'}
